<a href="https://colab.research.google.com/github/aradhana148/Medical-NER-RE/blob/master/medCAT_ct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install medcat
! pip install medcat~=1.16.0
# install seaborn
! pip install seaborn
try:
    from medcat.cat import CAT
except:
    print("WARNING: Runtime will restart automatically and please run other cells thereafter.")
    exit()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 47.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of thinc to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of thinc to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.0/269.0 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━

/usr/local/lib/python3.12/dist-packages/medcat/cat.py:17: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
import pandas as pd
import numpy as np
import pickle
import seaborn as sns

from matplotlib import pyplot as plt
from medcat.cat import CAT

/usr/local/lib/python3.12/dist-packages/medcat/cat.py:17: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
!rm -f models/medmen_wstatus_2021_oct.zip
!mkdir -p models

# Download from the updated location (follows redirects)
!wget -q -L -O models/medmen_wstatus_2021_oct.zip \
  https://cogstack-medcat-example-models.s3.eu-west-2.amazonaws.com/medcat-example-models/medmen_wstatus_2021_oct.zip

# Sanity checks
!ls -lh models/medmen_wstatus_2021_oct.zip
!file models/medmen_wstatus_2021_oct.zip
!python - << 'PY'
import zipfile
p="models/medmen_wstatus_2021_oct.zip"
print("is_zipfile:", zipfile.is_zipfile(p))


-rw-r--r-- 1 root root 536M Aug 25  2023 models/medmen_wstatus_2021_oct.zip
models/medmen_wstatus_2021_oct.zip: Zip archive data, at least v2.0 to extract, compression method=store
/bin/bash: line 1: warning: here-document at line 1 delimited by end-of-file (wanted `PY')
is_zipfile: True


In [ ]:
cat = CAT.load_model_pack("models/medmen_wstatus_2021_oct.zip")

/usr/local/lib/python3.12/dist-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_md' (3.1.0) was trained with spaCy v3.1.0 and may not be 100% compatible with the current version (3.7.5). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
INFO:medcat.cat:{
  "Model ID": null,
  "Last Modified On": null,
  "History (from least to most recent)": [],
  "Description": "No description",
  "Source Ontology": null,
  "Location": null,
  "MetaCAT models": {},
  "Basic CDB Stats": {},
  "Performance": {
    "ner": {},
    "meta": {}
  },
  "Important Parameters (Partial view, all available in cat.config)": {
    "config.ner.min_name_len": {
      "value": 3,
      "description": "Minimum detection length (found terms/mentions shorter than this will not be detected)."
    },
    "config.ner.upper_ca

In [ ]:
import pandas as pd

df = pd.read_csv("/content/ct_data.csv")

clean = (
    df.iloc[:, 0]
      .astype(str)
      .str.replace(r"\s+", " ", regex=True)
      .str.strip()
      .reset_index(drop=True)
)



gt_concepts_col = df.iloc[:, 1].reset_index(drop=True)




In [ ]:
def medcat_pred_concepts(text: str):
    out = cat.get_entities(text)   # dict with "entities"
    ents = out.get("entities", {})

    preds = []
    for _, e in ents.items():
        # Prefer canonical name if present, else surface text
        name = (e.get("detected_name") or e.get("source_value") or "").strip().lower()
        if not name:
            continue

        # Negation/status: depends on model pack; medmen_wstatus includes status-type info
        # Common fields you may see: "meta_anns", "status", "negated" depending on version/pack.
        # We'll handle a few robustly.
        neg = False
        if "negated" in e:
            neg = bool(e["negated"])
        elif "status" in e and isinstance(e["status"], str):
            neg = e["status"].lower() in {"negated", "negative", "absent"}
        elif "meta_anns" in e and isinstance(e["meta_anns"], dict):
            # example-style: { "Status": { "value": "NEGATED", ... } }
            for k, v in e["meta_anns"].items():
                if isinstance(v, dict) and "value" in v and str(v["value"]).lower() in {"negated", "negative", "absent"}:
                    neg = True

        if neg and not name.startswith("no "):
            name = "no " + name

        preds.append(name)

    # de-duplicate, preserve order
    return list(dict.fromkeys(preds))


pred_concepts = [medcat_pred_concepts(t) for t in clean]
for i in range(5):
  print(clean[i])
  print(pred_concepts[i])
  print()


The air passages of the trachea, both main bronchi, lobar, and segmental bronchi are patent. There is an irregularly circumscribed nodular consolidation area adjacent to the diaphragm in the basal segment of the lower lobe of the right lung. A Reverse Halo sign is observed and is suspicious for invasive fungal infection. No involvement is observed in other parenchymal areas. No pleural effusion is detected. Heart size is within normal limits. No pericardial effusion is detected. Calibration of mediastinal major vascular structures is normal. Evaluation of mediastinal structures is suboptimal due to the absence of contrast material. Lymph nodes with diameters of 12 mm and 13 mm are observed in the carina and subcarinal areas. A central venous catheter is present. In the supraclavicular fossa and axilla, no lymph nodes of pathological size and appearance are observed. No lytic or destructive lesions are detected in the bone structures. No abnormalities are detected in the upper abdominal

In [ ]:
import ast

parsed_gt_concepts = []
for concepts_str in gt_concepts_col:
    # Convert string representation of list to actual list
    concepts_list = ast.literal_eval(concepts_str)

    # Clean each concept: lowercase and strip whitespace
    cleaned_concepts = [concept.lower().strip() for concept in concepts_list]
    parsed_gt_concepts.append(cleaned_concepts)

# Print the first 5 entries to verify
print("First 5 parsed and cleaned ground truth concepts:")
for i in range(5):
    print(parsed_gt_concepts[i])

First 5 parsed and cleaned ground truth concepts:
['patent air passages', 'nodular consolidation', 'reverse halo sign suspicious for invasive fungal infection', 'no pleural effusion', 'normal heart size', 'no pericardial effusion', 'normal mediastinal vascular structures', 'lymph nodes of 12mm and 13mm in carina', 'central venous catheter', 'no pathological lymph nodes in supraclavicular fossa and axilla', 'normal bone structures', 'no abnormalities in upper abdominal sections']
['patent trachea', 'patent bronchi', 'peribronchial thickening', 'luminal narrowing', 'mosaic attenuation pattern likely small airway disease', 'reticulonodular density', 'paraseptal emphysema', 'subcentimeter nonspecific pulmonary nodules', 'normal heart', 'normal mediastinal main vascular structures', 'no pericardial effusion', 'calcific atheroma plaques', 'normal esophagus', 'no enlarged lymph nodes', 'normal bone structures', 'normal upper abdominal organs', 'no liver lesions', 'normal bilateral adrenal gla

In [ ]:

import re

def align_concepts_with_gt(original_text: str, medcat_concepts: list, gt_concepts: list):
    final_concepts = []

    # Convert original_text to lowercase for case-insensitive matching
    text_lower = original_text.lower()

    # Prioritize ground truth concepts if they are present in the original text
    for gt_concept in gt_concepts:
        # Check for whole word match, case-insensitive
        # Escaping special characters in gt_concept to use in regex pattern
        pattern = r'\b' + re.escape(gt_concept) + r'\b'
        if re.search(pattern, text_lower):
            final_concepts.append(gt_concept)

    # Add unique MedCAT concepts, avoiding duplicates and substrings of already added concepts
    for medcat_concept in medcat_concepts:
        is_redundant = False
        # Check if it's already in final_concepts (exact match) or if it's a substring
        if medcat_concept in final_concepts:
            is_redundant = True
        else:
            for final_c in final_concepts:
                # Check if medcat_concept is a substring of an existing final_concept
                if medcat_concept in final_c and medcat_concept != final_c:
                    is_redundant = True
                    break
                # Check if an existing final_concept is a substring of medcat_concept
                if final_c in medcat_concept and medcat_concept != final_c:
                    # Decide which one to keep. For now, prioritize the longer, more specific one if GT is not involved.
                    # Since GT is prioritized, if final_c is from GT, keep it. If medcat_concept is longer and unique, add it.
                    pass # This logic can be refined, for now, simple check for existing substring covers basic redundancy.

        if not is_redundant:
            final_concepts.append(medcat_concept)

    return list(dict.fromkeys(final_concepts)) # Remove any accidental duplicates while preserving order


# Apply the function to the data
aligned_concepts = []
for i in range(len(clean)):
    aligned_concepts.append(align_concepts_with_gt(clean[i], pred_concepts[i], parsed_gt_concepts[i]))

# Print the first 5 aligned concepts to verify
print("First 5 aligned concepts:")
for i in range(5):
    print(f"Original Text: {clean[i]}")
    print(f"Aligned Concepts: {aligned_concepts[i]}")
    print() # Empty line for better readability

First 5 aligned concepts:
Original Text: The air passages of the trachea, both main bronchi, lobar, and segmental bronchi are patent. There is an irregularly circumscribed nodular consolidation area adjacent to the diaphragm in the basal segment of the lower lobe of the right lung. A Reverse Halo sign is observed and is suspicious for invasive fungal infection. No involvement is observed in other parenchymal areas. No pleural effusion is detected. Heart size is within normal limits. No pericardial effusion is detected. Calibration of mediastinal major vascular structures is normal. Evaluation of mediastinal structures is suboptimal due to the absence of contrast material. Lymph nodes with diameters of 12 mm and 13 mm are observed in the carina and subcarinal areas. A central venous catheter is present. In the supraclavicular fossa and axilla, no lymph nodes of pathological size and appearance are observed. No lytic or destructive lesions are detected in the bone structures. No abnormal

In [ ]:
print("First 5 entries showing original text, predicted concepts, parsed ground truth concepts, and final aligned concepts:")
for i in range(5):
    print(f"\n--- Entry {i+1} ---")
    print(f"Original Text: {clean[i]}")
    print(f"Predicted Concepts (MedCAT): {pred_concepts[i]}")
    print(f"Parsed Ground Truth Concepts: {parsed_gt_concepts[i]}")
    print(f"Final Aligned Concepts: {aligned_concepts[i]}")

First 5 entries showing original text, predicted concepts, parsed ground truth concepts, and final aligned concepts:

--- Entry 1 ---
Original Text: The air passages of the trachea, both main bronchi, lobar, and segmental bronchi are patent. There is an irregularly circumscribed nodular consolidation area adjacent to the diaphragm in the basal segment of the lower lobe of the right lung. A Reverse Halo sign is observed and is suspicious for invasive fungal infection. No involvement is observed in other parenchymal areas. No pleural effusion is detected. Heart size is within normal limits. No pericardial effusion is detected. Calibration of mediastinal major vascular structures is normal. Evaluation of mediastinal structures is suboptimal due to the absence of contrast material. Lymph nodes with diameters of 12 mm and 13 mm are observed in the carina and subcarinal areas. A central venous catheter is present. In the supraclavicular fossa and axilla, no lymph nodes of pathological size a

In [ ]:
def similarity2(a: str, b: str) -> float:
    a_tokens = set(a.lower().split())
    b_tokens = set(b.lower().split())

    if not a_tokens and not b_tokens:
        return 1.0
    if not a_tokens or not b_tokens:
        return 0.0

    intersection = a_tokens & b_tokens
    union = a_tokens | b_tokens

    return (2*len(intersection)) / (len(a_tokens)+len(b_tokens))


In [ ]:
def fuzzy_counts_one(gt_list, pred_list, threshold):
    gt = [g.strip().lower() for g in gt_list if g.strip()]
    pr = [p.strip().lower() for p in pred_list if p.strip()]

    used_gt = set()
    TP = 0

    for p in pr:
        best_j = None
        best_score = 0.0

        for j, g in enumerate(gt):
            if j in used_gt:
                continue

            score = similarity2(p, g)
            if score > best_score:
                best_score = score
                best_j = j

        if best_j is not None and best_score >= threshold:
            TP += 1
            used_gt.add(best_j)

    FP = len(pr) - TP
    FN = len(gt) - TP

    return TP, FP, FN


In [ ]:
def fuzzy_prf(gt_norm, pred_norm, threshold):
    TP = FP = FN = 0

    for gt, pr in zip(gt_norm, pred_norm):
        t, f, n = fuzzy_counts_one(gt, pr, threshold)
        TP += t
        FP += f
        FN += n

    precision = TP / (TP + FP) if TP + FP else 0
    recall    = TP / (TP + FN) if TP + FN else 0
    f1        = (2 * precision * recall / (precision + recall)) if precision + recall else 0

    return precision, recall, f1, (TP, FP, FN)


In [ ]:
for th in [0.5,0.6,0.7,0.75, 0.8, 0.85,0.9]:
    P, R, F1, counts = fuzzy_prf(parsed_gt_concepts, aligned_concepts, threshold=th)
    print(th, P, R, F1)

0.5 0.19153936545240893 0.6293436293436293 0.2936936936936937
0.6 0.1645123384253819 0.5405405405405406 0.25225225225225223
0.7 0.10928319623971798 0.3590733590733591 0.16756756756756758
0.75 0.10928319623971798 0.3590733590733591 0.16756756756756758
0.8 0.10928319623971798 0.3590733590733591 0.16756756756756758
0.85 0.10928319623971798 0.3590733590733591 0.16756756756756758
0.9 0.10928319623971798 0.3590733590733591 0.16756756756756758
